In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv("final_post1.csv")



In [2]:
import string
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.cross_validation import KFold

lemmatizer = WordNetLemmatizer()

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

## Removing HTML tags
def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

## Removing URL
def strip_URL(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)

## Removing Hashtags and @tags 
def strip_all_tags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

## Removing special characters 
def strip_special_chars(text):
    return re.sub('\W+',' ', text)

## Removing extra white spaces
def strip_extra_white_spaces(text):
    return re.sub(' +',' ',text).strip()

## Removing extra new lines
def strip_extra_new_lines(text):
    return re.sub("\n+","\n", text)

def inverse_transform(l):
        sent = ''
        for word in l:
            sent += word+" "
        return [sent]

def tokenize(document):
    document = cleanhtml(document)
    document = strip_URL(document)
    document = strip_all_tags(document)
    document = strip_special_chars(document)
    document = strip_extra_white_spaces(document)
    document = strip_extra_new_lines(document)
    
    stopwords_custom = pd.read_csv("stopwords1.csv")
    stopwords_custom = stopwords_custom.ix[:,0].map(lambda x:x.lower())
        
    # Break the document into sentences
    for sent in sent_tokenize(document):
        # Break the sentence into part of speech tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Apply preprocessing to the token
            token = token.lower() 

            # If stopword, ignore token and continue
            if token.lower in stopwords_custom.tolist():
                continue

            # If punctuation, ignore token and continue
            if all(char in string.punctuation for char in token):
                continue
            
            yield token

            # Lemmatize the token and yield
#             lemma = lemmatize(token, tag)
#             yield lemma

def lemmatize(token, tag):
    tag = {
        'N': wn.NOUN,
        'V': wn.VERB,
       # 'R': wn.ADV,
        #'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    return lemmatizer.lemmatize(token, tag)
    
def preprocessing(X):
    tokens_dict = {}
    count = 0
    for doc in X:
        tokens = list(tokenize(doc))
        tokens_dict[count] = tokens
#         tokens_dict[count] = inverse_transform(tokens)
        count += 1
    return tokens_dict

In [3]:
temporal_list = pd.read_csv("temporal_words.csv")
temporal_list = temporal_list.ix[:,0].map(lambda x:x.lower())
def TemporalEntitiesCount(X):
    count_list = []
    for document in X:
        count = 0
        # Break the document into sentences
        for sent in sent_tokenize(document):
            # Break the sentence into part of speech tagged tokens
            for token in wordpunct_tokenize(sent):
                if token in temporal_list.tolist():
                    count += 1 
        count_list.append(count)
#     return np.asarray(count_list)
    return np.reshape(np.asarray(count_list),(len(X),1))

In [4]:
from sklearn.cross_validation import train_test_split as tts
from scipy import sparse

X = data["posts_p"]
y = data["Tag"]

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.3, random_state=111)
preprocessed_train = preprocessing(X_train)

tfidf = TfidfVectorizer(tokenizer=identity, preprocessor=None, lowercase=False, use_idf=False)
tfs = tfidf.fit_transform(preprocessed_train.values())

# TempCount = TemporalEntitiesCount(X_train)


In [1]:
# from scipy.sparse import hstack
# feature = hstack((tfs,TempCount))
feature = tfs
type(tfs)

NameError: name 'tfs' is not defined

In [6]:
classifier = SGDClassifier()
model = classifier.fit(feature, y_train)

preprocessed_test = preprocessing(X_test)
tfs_test = tfidf.transform(preprocessed_test.values())

TempCount_test = TemporalEntitiesCount(X_test)
print("SGD Classifier")

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()
feature_test = tfs_test

y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

SGD Classifier
             precision    recall  f1-score   support

          0       0.77      0.80      0.78        84
          1       0.73      0.70      0.71        66

avg / total       0.75      0.75      0.75       150



In [7]:
classifier = LinearSVC()
model = classifier.fit(feature, y_train)

preprocessed_test = preprocessing(X_test)
tfs_test = tfidf.transform(preprocessed_test.values())

TempCount_test = TemporalEntitiesCount(X_test)
print("SVM classifier")

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()
feature_test = tfs_test

y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

SVM classifier
             precision    recall  f1-score   support

          0       0.81      0.77      0.79        84
          1       0.73      0.77      0.75        66

avg / total       0.78      0.77      0.77       150



In [8]:
classifier = MultinomialNB()
model = classifier.fit(feature, y_train)

preprocessed_test = preprocessing(X_test)
tfs_test = tfidf.transform(preprocessed_test.values())

TempCount_test = TemporalEntitiesCount(X_test)
print("Naive Bayes Classifier")

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()
feature_test = tfs_test

y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

Naive Bayes Classifier
             precision    recall  f1-score   support

          0       0.82      0.88      0.85        84
          1       0.83      0.76      0.79        66

avg / total       0.83      0.83      0.83       150

